In [ ]:
import requests

In [ ]:
ENDPOINT = 'https://en.wikipedia.org/w/api.php'

In [ ]:

parameters = { 'action' : 'query',
               'prop' : 'revisions',
               'format' : 'json',
               'rvdir' : 'older',
              'titles' : 'Panama Papers',
              'rvlimit' : 'max',
               'continue' : '' }

Look at the response: see `continue`. This means there are more revisions that are not in the response, but we can get them by loading results one page at a time.

In [ ]:
wp_call = requests.get(ENDPOINT, params=parameters)
response = wp_call.json()
print(response)

We need to stop querying the API once it has no more results for us. We will use the `done` variable to keep track. In this example we'll keep track of how many times we call the api in `iteration` and the total number of revisions we've seen in `num_revisions`. 

In [ ]:
done = False
iteration = 0
num_revisions = 0

The cell below shows how we page through results.  We just check if the 'continue' key is in the `response` dictionary.  If it is, then we need to  assign the 'continue' and 'rvcontinue' api parameters to the values the API is giving us.  This tells the API to start sending us results from where we left off.  You don't need to remember the details of how this works, but it is important to know that APIs often follow this kind of 'pagination' pattern. 

In [ ]:
while not done:
    wp_call = requests.get(ENDPOINT, params=parameters)
    response = wp_call.json()

    if 'continue' in response:
        # we need the value of rvcontinue to page through results
        parameters['continue']  = response['continue']['continue']
        parameters['rvcontinue'] = response['continue']['rvcontinue']
    else:
        done = True
        
    query = response['query']
    pages = query['pages']
    
    # get the page
    page_keys = list(pages.keys())
    page_key = page_keys[0]
    page = pages[page_key]

    
    # now we need to loop over the revisions
    revisions = page['revisions']
    num_revisions = num_revisions + len(revisions)
        
    print(f'iteration: {iteration}, num_revisions: {num_revisions}')
    iteration = iteration + 1